## Learning FourSquare API with Python

### Introduction:

In this lab, you will learn in details how to make calls to the Foursquare API for different purposes. You will learn how to construct a URL to send a request to the API to search for a specific type of venues, to explore a particular venue.

In [1]:
pip install python-dotenv

^C
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
from dotenv import load_dotenv
from geopy.geocoders import Nominatim   # module to convert an address into latitude and longitude values
import requests
from pandas import json_normalize   # tranforming json file into a pandas dataframe 

In [4]:
# Define the content of the .env file
env_content = """
DB_ID=UWRB20B2EYP153CFGL0K3SJLNL0LGBIY5RSYFO3X2CNAPUC4
DB_SECRET=E4MC1PE4CLKJZZ0ZC3E1MZ2ZV4R2X2BWZ334UYPARUAJADH2
"""

# Specify the path to the .env file
env_file_path = '.env'

# Write the content to the .env file
with open(env_file_path, 'w') as env_file:
    env_file.write(env_content)

# Print a message indicating that the file has been created
print(f".env file created at {os.path.abspath(env_file_path)}")


.env file created at C:\Users\raopo\Desktop\Projects\Data_Science\.env


In [5]:
# Load variables from .env file
load_dotenv()

True

In [6]:
# Access environment variables
db_username = os.environ.get('DB_ID')
db_password = os.environ.get('DB_SECRET')

print("DB_ID:", db_username)
print("DB_SECRET:", db_password)


DB_ID: UWRB20B2EYP153CFGL0K3SJLNL0LGBIY5RSYFO3X2CNAPUC4
DB_SECRET: E4MC1PE4CLKJZZ0ZC3E1MZ2ZV4R2X2BWZ334UYPARUAJADH2


#### Define Foursquare Credentials and Version

In [7]:
CLIENT_ID = os.environ.get('DB_ID') # your Foursquare ID
CLIENT_SECRET = os.environ.get('DB_SECRET') # your Foursquare Secret
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: UWRB20B2EYP153CFGL0K3SJLNL0LGBIY5RSYFO3X2CNAPUC4
CLIENT_SECRET:E4MC1PE4CLKJZZ0ZC3E1MZ2ZV4R2X2BWZ334UYPARUAJADH2


In [8]:
address = '102 North End Ave, New York, NY'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.7149555 -74.0153365


## 1. Search for a specific venue category

Now, let's assume that it is lunch time, and you are craving Italian food. So, let's define a query to search for Italian food that is within 500 metres from the Conrad Hotel.

In [9]:
search_query = 'italian'
radius = 500
ll = f'{latitude},{longitude}'

#### Define the URL

In [40]:
base_url = "https://api.foursquare.com/v3/places/search"
url = f"{base_url}?query={search_query}&radius={radius}&ll={ll}"

headers = {
    "accept": "application/json",
    "Authorization": "fsq3slWZEeFW743yx8er5OvDF1NCO2Z5R8qnD2QTiYii1sA="
}

result = requests.get(url, headers=headers)

data = result.json()
data

{'results': [{'fsq_id': '5d41d093104e90000882db55',
   'categories': [{'id': 13003,
     'name': 'Bar',
     'short_name': 'Bar',
     'plural_name': 'Bars',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/pub_',
      'suffix': '.png'}},
    {'id': 13236,
     'name': 'Italian Restaurant',
     'short_name': 'Italian',
     'plural_name': 'Italian Restaurants',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/italian_',
      'suffix': '.png'}}],
   'chains': [],
   'closed_bucket': 'Unsure',
   'distance': 171,
   'geocodes': {'main': {'latitude': 40.713759, 'longitude': -74.014339},
    'roof': {'latitude': 40.713759, 'longitude': -74.014339}},
   'link': '/v3/places/5d41d093104e90000882db55',
   'location': {'address': '200 Vesey St',
    'census_block': '360610317031002',
    'country': 'US',
    'cross_street': 'West St',
    'dma': 'New York',
    'formatted_address': '200 Vesey St (West St), New York, NY 10281',
    'locality': 'New

#### Get relevant part of JSON and transform it into a pandas dataframe

In [11]:
# assign relevant part of JSON to venues
venues = data['results']

# tranform venues into a dataframe
df = json_normalize(venues)
df.head()

,fsq_id,categories,chains,closed_bucket,distance,link,name,timezone,geocodes.main.latitude,geocodes.main.longitude,...,location.country,location.cross_street,location.dma,location.formatted_address,location.locality,location.postcode,location.region,geocodes.drop_off.latitude,geocodes.drop_off.longitude,location.address_extended
0,5d41d093104e90000882db55,"[{'id': 13003, 'name': 'Bar', 'short_name': 'B...",[],Unsure,171,/v3/places/5d41d093104e90000882db55,Sant Ambroeus Brookfield,America/New_York,40.713759,-74.014339,...,US,West St,New York,"200 Vesey St (West St), New York, NY 10281",New York,10281,NY,NaN,NaN,NaN
1,60eda733a7ddae4973db6ee8,"[{'id': 13236, 'name': 'Italian Restaurant', '...",[],VeryLikelyOpen,181,/v3/places/60eda733a7ddae4973db6ee8,Cicci Di Carne,America/New_York,40.713984,-74.015963,...,US,Suite 107B,New York,"250 Vesey St (Suite 107B), New York, NY 10281",New York,10281,NY,NaN,NaN,NaN
2,63d58772256bdb3de18e26dd,"[{'id': 13236, 'name': 'Italian Restaurant', '...",[],LikelyOpen,188,/v3/places/63d58772256bdb3de18e26dd,St Ambroeus,America/New_York,40.713857,-74.014692,...,US,,New York,"200 Vesey St, New York, NY 10281",New York,10281,NY,40.713967,-74.014647,NaN
3,3fd66200f964a520aae61ee3,"[{'id': 13064, 'name': 'Pizzeria', 'short_name...",[],LikelyOpen,470,/v3/places/3fd66200f964a520aae61ee3,Gigino Trattoria,America/New_York,40.717261,-74.010506,...,US,btw Duane & Reade Sts.,New York,"323 Greenwich St (btw Duane & Reade Sts.), New...",New York,10013,NY,40.717301,-74.010671,Frnt 1
4,592b5e9c0ff4f9137e2a8390,"[{'id': 13025, 'name': 'Wine Bar', 'short_name...",[],LikelyOpen,490,/v3/places/592b5e9c0ff4f9137e2a8390,L' Angolo Tribeca,America/New_York,40.717319,-74.010378,...,US,Greenwich,New York,"190A Duane St (Greenwich), New York, NY 10013",New York,10013,NY,40.717433,-74.010365,NaN


#### Define information of interest and filter dataframe

In [20]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['fsq_id' ,'name'] + [col for col in df.columns if col.startswith('location.')]
df_filtered = df.loc[:, filtered_columns]
# df_filtered

def get_category_type(row):
    categories_list = row['categories']
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# Apply the get_category_type function to each row to extract the category
df_filtered['categories'] = df.apply(get_category_type, axis=1)

# Clean column names by keeping only the last term
df_filtered.columns = [column.split('.')[-1] for column in df_filtered.columns]

# Display the filtered DataFrame
df_filtered

,fsq_id,name,address,census_block,country,cross_street,dma,formatted_address,locality,postcode,region,address_extended,categories
0,5d41d093104e90000882db55,Sant Ambroeus Brookfield,200 Vesey St,360610317031002,US,West St,New York,"200 Vesey St (West St), New York, NY 10281",New York,10281,NY,NaN,Bar
1,60eda733a7ddae4973db6ee8,Cicci Di Carne,250 Vesey St,360610317031002,US,Suite 107B,New York,"250 Vesey St (Suite 107B), New York, NY 10281",New York,10281,NY,NaN,Italian Restaurant
2,63d58772256bdb3de18e26dd,St Ambroeus,200 Vesey St,360610317031002,US,,New York,"200 Vesey St, New York, NY 10281",New York,10281,NY,NaN,Italian Restaurant
3,3fd66200f964a520aae61ee3,Gigino Trattoria,323 Greenwich St,360610039003010,US,btw Duane & Reade Sts.,New York,"323 Greenwich St (btw Duane & Reade Sts.), New...",New York,10013,NY,Frnt 1,Pizzeria
4,592b5e9c0ff4f9137e2a8390,L' Angolo Tribeca,190A Duane St,360610039003010,US,Greenwich,New York,"190A Duane St (Greenwich), New York, NY 10013",New York,10013,NY,NaN,Wine Bar
5,d462e1585b104ed413c3292d,iPizzaNY,147 Chambers St,360610021003000,US,NaN,New York,"147 Chambers St, New York, NY 10007",New York,10007,NY,NaN,Italian Restaurant
6,95e98b4cb7e64cfc140c5d29,Global Cuisine,222 Murray St,360610317033007,US,NaN,New York,"222 Murray St, New York, NY 10282",New York,10282,NY,NaN,Chinese Restaurant
7,621bd89fdda85a16a2075f0c,Sauce pizzeria,225 Liberty St,360610317031002,US,,New York,"225 Liberty St, New York, NY 10281",New York,10281,NY,NaN,Pizzeria
8,3a083b30a7744cb94f3fe6da,Cafe Amore's Restaurant,147 Chambers St,360610021003000,US,NaN,New York,"147 Chambers St, New York, NY 10007",New York,10007,NY,NaN,Pizzeria


## 2. Explore a Given Venue

### A. Let's explore the closest Italian restaurant -- Cafe Amore's Restaurant	

In [42]:
venue_id = '60eda733a7ddae4973db6ee8'

base_url = "https://api.foursquare.com/v3/places/"
url = url = f"{base_url}{venue_id}"

headers = {
    "accept": "application/json",
    "Authorization": "fsq3slWZEeFW743yx8er5OvDF1NCO2Z5R8qnD2QTiYii1sA="
}

response = requests.get(url, headers=headers)
data = response.json()
data


{'fsq_id': '60eda733a7ddae4973db6ee8',
 'categories': [{'id': 13236,
   'name': 'Italian Restaurant',
   'short_name': 'Italian',
   'plural_name': 'Italian Restaurants',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/italian_',
    'suffix': '.png'}},
  {'id': 13334,
   'name': 'Sandwich Spot',
   'short_name': 'Sandwich Spot',
   'plural_name': 'Sandwich Spots',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/deli_',
    'suffix': '.png'}}],
 'chains': [],
 'closed_bucket': 'VeryLikelyOpen',
 'geocodes': {'main': {'latitude': 40.713984, 'longitude': -74.015963},
  'roof': {'latitude': 40.713984, 'longitude': -74.015963}},
 'link': '/v3/places/60eda733a7ddae4973db6ee8',
 'location': {'address': '250 Vesey St',
  'census_block': '360610317031002',
  'country': 'US',
  'cross_street': 'Suite 107B',
  'dma': 'New York',
  'formatted_address': '250 Vesey St (Suite 107B), New York, NY 10281',
  'locality': 'New York',
  'postcode': '10281',
  'region'

In [34]:
data.keys()

dict_keys(['fsq_id', 'categories', 'chains', 'closed_bucket', 'geocodes', 'link', 'location', 'name', 'related_places', 'timezone'])

####  Get the venue's overall rating

In [49]:
try:
    rating = data.get('rating')
    if rating is not None:
        print("Rating:", rating)
    else:
        print("This venue has not been rated yet.")
except KeyError:
    print("This venue has not been rated yet.")

This venue has not been rated yet.
